In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

## Read in data

In [4]:
#Import Joshua Tree routes
jtree_boulder = pd.read_csv('jtree_boulder.csv')
jtree_rock = pd.read_csv('CA_JTree_Routes_Rock_250.csv')
#Import Red Rocks routes
redrocks_boulder = pd.read_csv('redrocks_boulder.csv')
redrocks_rock = pd.read_csv('redrocks_rock.csv')
#Import CA Aid Routes
ca_aid = pd.read_csv('ca_aid.csv')
#Import Sierra East Routes
sierraeast_boulder = pd.read_csv('sierra_east_boulder.csv')
sierraeast_rock = pd.read_csv('sierra_east_rock.csv')
#Import Yosemite Routes
yosemite_boulder = pd.read_csv('yosemite_boulder.csv')
yosemite_rock = pd.read_csv('yosemite_rock.csv')

frames = [jtree_boulder, jtree_rock, redrocks_boulder, redrocks_rock, ca_aid, 
          sierraeast_boulder, sierraeast_rock, yosemite_boulder, yosemite_rock]

df = pd.concat(frames)

df.head()

,name,rating,votes,grade,type,height,pitches,safety,commitment,state,sub_area,photos,comments,near_grades,aid_grade
0,Todd Gordon Memorial Chossy Pile of Radness (a...,2.5,6,5.11b,"['Sport', 'TR', 'Boulder']",35.0,1,NaN,NaN,California,Rattlesnake Canyon,2,7,"['5.10a', '5.9', '5.9', '5.11a/b', '5.7', 'V2+']",NaN
1,A Hearty Handful of Cheek,2.0,2,V0,['Boulder'],6.0,1,PG13,NaN,California,Group Campsites Rd,1,0,[],NaN
2,Gait of Power Traverse,2.0,1,V1,['Boulder'],20.0,1,NaN,NaN,California,Indian Cove Cam…,0,0,"['5.9', '5.10a', '5.11c', 'V1-2', '5.8-', '5.1...",NaN
3,Crack Alley,2.0,8,V1-2,['Boulder'],15.0,1,NaN,NaN,California,Indian Cove Cam…,3,0,"['5.9', '5.10a', '5.11c', '5.8-', '5.10a/b', '...",NaN
4,Fart Man,2.0,2,V1,['Boulder'],15.0,1,NaN,NaN,California,Indian Cove Cam…,0,0,"['5.5', '5.9', '5.8', '5.5', '5.0', '5.9', '5....",NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5985 entries, 0 to 989
Data columns (total 15 columns):
name           5985 non-null object
rating         5985 non-null float64
votes          5985 non-null int64
grade          5979 non-null object
type           5985 non-null object
height         5291 non-null float64
pitches        5985 non-null int64
safety         735 non-null object
commitment     414 non-null object
state          5985 non-null object
sub_area       5985 non-null object
photos         5985 non-null int64
comments       5985 non-null int64
near_grades    5985 non-null object
aid_grade      170 non-null object
dtypes: float64(2), int64(4), object(9)
memory usage: 748.1+ KB


In [6]:
#Shuffle the data frame. Frac=1 means all rows
df = df.sample(frac=1).reset_index(drop=True)

df.head()

,name,rating,votes,grade,type,height,pitches,safety,commitment,state,sub_area,photos,comments,near_grades,aid_grade
0,Going Both Ways,3.5,17,5.11c,"['Sport', 'Alpine']",60.0,1,NaN,NaN,California,Dike Wall,1,0,"['5.11a', '5.11d', '5.11b', '5.11b', '5.10c', ...",NaN
1,Too Close for Comfort,2.0,2,5.11d,['Sport'],40.0,1,NaN,NaN,California,Main Island,0,0,"['5.10a', '5.10a', '5.6', '5.8', '5.10b', '5.1...",NaN
2,Victim of AIDS,3.1,22,5.12a,"['Trad', 'TR', 'Aid']",25.0,1,NaN,NaN,California,Mt Woodson,1,5,"['5.8', '5.7', '5.8', '5.10a']",NaN
3,Christmas in July,2.4,8,5.10b/c,['Sport'],100.0,1,NaN,NaN,California,Rock Creek,1,0,"['5.10', '5.10c', '5.10d', '5.11', '5.10b', '5...",NaN
4,Bridge of Sighs,2.8,4,5.11a,['Trad'],50.0,1,R,NaN,California,Rattlesnake Canyon,0,1,[],NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5985 entries, 0 to 5984
Data columns (total 15 columns):
name           5985 non-null object
rating         5985 non-null float64
votes          5985 non-null int64
grade          5979 non-null object
type           5985 non-null object
height         5291 non-null float64
pitches        5985 non-null int64
safety         735 non-null object
commitment     414 non-null object
state          5985 non-null object
sub_area       5985 non-null object
photos         5985 non-null int64
comments       5985 non-null int64
near_grades    5985 non-null object
aid_grade      170 non-null object
dtypes: float64(2), int64(4), object(9)
memory usage: 701.5+ KB


## Pre-Processing

### Cleaning: Removing Duplicates, Outliers, Unnecessary Data, Etc.

In [8]:
#Remove duplicates
def remove_duplicates(df):
    df.drop_duplicates(keep = 'first',inplace=True)

remove_duplicates(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5970 entries, 0 to 5984
Data columns (total 15 columns):
name           5970 non-null object
rating         5970 non-null float64
votes          5970 non-null int64
grade          5964 non-null object
type           5970 non-null object
height         5278 non-null float64
pitches        5970 non-null int64
safety         730 non-null object
commitment     409 non-null object
state          5970 non-null object
sub_area       5970 non-null object
photos         5970 non-null int64
comments       5970 non-null int64
near_grades    5970 non-null object
aid_grade      166 non-null object
dtypes: float64(2), int64(4), object(9)
memory usage: 746.2+ KB


In [9]:
df.rating.describe()

count    5970.000000
mean        2.291055
std         0.875460
min         0.000000
25%         1.800000
50%         2.300000
75%         3.000000
max         4.000000
Name: rating, dtype: float64

In [10]:
df.rating.isna().sum()

0

In [11]:
df.votes.describe()

count    5970.000000
mean       17.683250
std        57.647347
min         0.000000
25%         1.000000
50%         3.000000
75%        12.000000
max      1469.000000
Name: votes, dtype: float64

In [12]:
#Remove voting outliers
def remove_outliers(df):
    #Remove rows with over 1000 votes
    high_votes = (df.loc[df.votes > 500]).index
    df.drop(index = high_votes, inplace = True)
    
    #Remove rows with less than 1 vote
    low_votes = (df.loc[df.votes < 1]).index
    df.drop(index = low_votes, inplace = True)    
    
remove_outliers(df)
df.describe()

,rating,votes,height,pitches,photos,comments
count,5899.000000,5899.000000,5236.000000,5899.000000,5899.000000,5899.000000
mean,2.311341,16.071877,135.691367,1.628412,2.057128,1.780641
std,0.848068,40.084980,319.343940,2.456039,4.289059,4.023586
min,0.000000,1.000000,4.000000,1.000000,0.000000,0.000000
25%,1.800000,1.000000,15.000000,1.000000,0.000000,0.000000
50%,2.300000,3.000000,50.000000,1.000000,1.000000,0.000000
75%,3.000000,12.000000,100.000000,1.000000,2.000000,2.000000
max,4.000000,496.000000,4700.000000,35.000000,116.000000,72.000000


In [13]:
def remove_noGrade(df):
    """Remove rows that have no grade listed."""
    df.dropna(subset=['grade'], inplace = True)

remove_noGrade(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5893 entries, 0 to 5984
Data columns (total 15 columns):
name           5893 non-null object
rating         5893 non-null float64
votes          5893 non-null int64
grade          5893 non-null object
type           5893 non-null object
height         5230 non-null float64
pitches        5893 non-null int64
safety         720 non-null object
commitment     398 non-null object
state          5893 non-null object
sub_area       5893 non-null object
photos         5893 non-null int64
comments       5893 non-null int64
near_grades    5893 non-null object
aid_grade      158 non-null object
dtypes: float64(2), int64(4), object(9)
memory usage: 736.6+ KB


In [14]:
def remove_noHeight(df):
    """Remove rows that have no height."""
    df.dropna(subset=['height'], inplace = True)

remove_noHeight(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5230 entries, 0 to 5984
Data columns (total 15 columns):
name           5230 non-null object
rating         5230 non-null float64
votes          5230 non-null int64
grade          5230 non-null object
type           5230 non-null object
height         5230 non-null float64
pitches        5230 non-null int64
safety         667 non-null object
commitment     357 non-null object
state          5230 non-null object
sub_area       5230 non-null object
photos         5230 non-null int64
comments       5230 non-null int64
near_grades    5230 non-null object
aid_grade      143 non-null object
dtypes: float64(2), int64(4), object(9)
memory usage: 653.8+ KB


In [15]:
def wrong_commitment(row):
    """Remove rows where a boulder has a commitment grade 'V'."""
    num_types = len(row['type'].split())
    name_type = row['type'].split()[0].replace('[', '').replace("'", '').replace(']', '')
    if (row['commitment'] == 'V') and (num_types == 1) and (name_type == 'Boulder'):
        df.drop(index = row.name, inplace = True)
    
df.apply(lambda row: wrong_commitment(row), axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5227 entries, 0 to 5984
Data columns (total 15 columns):
name           5227 non-null object
rating         5227 non-null float64
votes          5227 non-null int64
grade          5227 non-null object
type           5227 non-null object
height         5227 non-null float64
pitches        5227 non-null int64
safety         666 non-null object
commitment     354 non-null object
state          5227 non-null object
sub_area       5227 non-null object
photos         5227 non-null int64
comments       5227 non-null int64
near_grades    5227 non-null object
aid_grade      143 non-null object
dtypes: float64(2), int64(4), object(9)
memory usage: 653.4+ KB


In [16]:
def split_it(row):
    """Standardize type column by removing brackets, spaces, and quotation marks"""
    split_row = row['type'].replace('[', '').replace(']', '').replace("'", '').replace(' ', '')
    return split_row.split(',')

df['type'] = df.apply(lambda row: split_it(row), axis=1)

### Converting Categorical Data to Numerical Data

In [17]:
import re

In [18]:
#Calculate Popularity
def popularity2(row):
    """
    bayesian rating = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C 
    where:
    R = average for the anime
    v = number of votes for the anime
    m = minimum votes required to be listed (currently 12)
    C = the mean vote across the whole report (currently 7.5017)
    source: https://www.animenewsnetwork.com/encyclopedia/ratings-anime.php#best_bayesian
    """
    R = row.rating 
    C = df.rating.mean()
    V = row.votes
    M = 1
    popularity = ((V / (V+M)) * R) + (M / (V+M)) * C
    return popularity

#Apply the popularity function to every row and add it to a column named 'popularity'
df['popularity'] = df.apply(lambda row: popularity2(row), axis=1)
df.popularity.describe()

count    5227.000000
mean        2.342199
std         0.622392
min         0.698971
25%         1.848713
50%         2.298713
75%         2.749485
max         3.996053
Name: popularity, dtype: float64

In [19]:
#Calculate Difficulty
def combined_difficulty(row):
    """
    Integer to represent general climbing difficulty, combining all types of climbing.
    Source: https://rockclimbingforwomen.com/climbing-grade-comparison-bouldering-vs-sport-climbing-equivalent
    """
    if type(row) == str:
        grade = row
    else:
        grade = row.grade
    if(grade == None):
        difficulty = None
        return difficulty
    elif (re.search('5.15d', grade) or re.search('V16', grade)):
        difficulty = 18
    elif ( re.search('5.15c', grade) or re.search('V15', grade)):
        difficulty = 17
    elif (re.search('5.15b', grade) or re.search('V14', grade)):
        difficulty = 16
    elif (re.search('V15', grade)):
        difficulty = 15
    elif (re.search('5.15a', grade) or re.search('V12', grade)):
        difficulty = 14
    elif (re.search('5.14d', grade) or re.search('V11', grade)):
        difficulty = 13
    elif (
        re.search('5.14b', grade) or re.search('5.14c', grade) or 
        re.search('V10', grade) or re.search('5.14+', grade)
    ):
        difficulty = 12
    elif (re.search('5.14a', grade) or re.search('V9', grade)):
        difficulty = 11
    elif (re.search('5.13d', grade) or re.search('V8', grade)):
        difficulty = 10
    elif (re.search('5.13c', grade) or re.search('V7', grade)):
        difficulty = 9
    elif (
            re.search('5.13a', grade) or re.search('5.13b', grade) or 
            re.search('V6', grade) or re.search('5.13-', grade)
    ):
        difficulty = 8
    elif (
            re.search('5.12a', grade) or re.search('5.12b', grade) or 
            re.search('V4', grade) or re.search('5.12-', grade)
    ):
        difficulty = 6    
    elif (
            re.search('5.12c', grade) or re.search('5.12d', grade) or 
            re.search('V5', grade) or re.search('5.12+', grade)
    ):
        difficulty = 7
    elif (
            re.search('5.11a', grade) or re.search('5.11b', grade) or 
            re.search('V2', grade) or re.search('5.11-', grade)
    ):
        difficulty = 4    
    elif (
            re.search('5.11c', grade) or re.search('5.11d', grade) or 
            re.search('V3', grade) or re.search('5.11+', grade)
    ):
        difficulty = 5

    elif (
            re.search('5.7', grade) or re.search('5.8', grade) or 
            re.search('5.9', grade) or re.search('5.10a', grade) or 
            re.search('5.10b', grade) or re.search('V0', grade) or
            re.search('5.10-', grade)
    ):
        difficulty = 2
    elif (
            re.search('5.10c', grade) or re.search('5.10d', grade) or 
            re.search('V1', grade) or re.search('5.10+', grade)
    ):
        difficulty = 3
    elif (
            re.search('5.0', grade) or re.search('5.1', grade) or 
            re.search('5.2', grade) or re.search('5.3', grade) or 
            re.search('5.4', grade) or re.search('5.5', grade) or
            re.search('5.6', grade) or re.search('VB', grade) or
            re.search('V-easy', grade) or re.search('3rd', grade) or
            re.search('4th', grade) or re.search('5th', grade) or
            re.search('Easy', grade)
    ):
        difficulty = 1
    else:
        difficulty = None

    return difficulty

df['combined_difficulty'] = df.apply(lambda row: combined_difficulty(row), axis=1)
df.head()

,name,rating,votes,grade,type,height,pitches,safety,commitment,state,sub_area,photos,comments,near_grades,aid_grade,popularity,combined_difficulty
0,Going Both Ways,3.5,17,5.11c,"[Sport, Alpine]",60.0,1,NaN,NaN,California,Dike Wall,1,0,"['5.11a', '5.11d', '5.11b', '5.11b', '5.10c', ...",NaN,3.433047,5
1,Too Close for Comfort,2.0,2,5.11d,[Sport],40.0,1,NaN,NaN,California,Main Island,0,0,"['5.10a', '5.10a', '5.6', '5.8', '5.10b', '5.1...",NaN,2.098285,5
2,Victim of AIDS,3.1,22,5.12a,"[Trad, TR, Aid]",25.0,1,NaN,NaN,California,Mt Woodson,1,5,"['5.8', '5.7', '5.8', '5.10a']",NaN,3.064994,6
3,Christmas in July,2.4,8,5.10b/c,[Sport],100.0,1,NaN,NaN,California,Rock Creek,1,0,"['5.10', '5.10c', '5.10d', '5.11', '5.10b', '5...",NaN,2.388317,2
4,Bridge of Sighs,2.8,4,5.11a,[Trad],50.0,1,R,NaN,California,Rattlesnake Canyon,0,1,[],NaN,2.698971,4


In [20]:
#Calculate Difficulty
def difficulty_sport_trad(row):
    """
    Integer to represent general climbing difficulty, combining all types of climbing.
    Source: https://rockclimbingforwomen.com/climbing-grade-comparison-bouldering-vs-sport-climbing-equivalent
    """
    grade = row.grade
    if (re.search('5.15', grade)):
        difficulty = 22
    elif (re.search('5.14', grade)):
        difficulty = 21
    elif (re.search('5.13c', grade)):
        difficulty = 20
    elif (re.search('5.13b', grade)):
        difficulty = 19
    elif (re.search('5.13a', grade) or re.search('5.13', grade)):      
        difficulty = 18
    elif (re.search('5.12c', grade)):
        difficulty = 17
    elif (re.search('5.12b', grade)):
        difficulty = 16
    elif (re.search('5.12a', grade) or re.search('5.12', grade)):      
        difficulty = 15
    elif (re.search('5.11d', grade)):
        difficulty = 14
    elif (re.search('5.11c', grade)):
        difficulty = 13
    elif (re.search('5.11b', grade)):
        difficulty = 12
    elif (re.search('5.11a', grade) or re.search('5.11', grade)):
        difficulty = 11
    elif (re.search('5.10d', grade)):
        difficulty = 10
    elif (re.search('5.10c', grade)):
        difficulty = 9
    elif (re.search('5.10b', grade)):
        difficulty = 8
    elif (re.search('5.10a', grade) or re.search('5.10', grade)):
        difficulty = 7
    elif (re.search('5.9', grade)):
        difficulty = 6
    elif (re.search('5.8', grade)):
        difficulty = 5
    elif (re.search('5.7', grade)):
        difficulty = 4
    elif (re.search('5.6', grade)):
        difficulty = 3
    elif (re.search('5.5', grade)):
        difficulty = 2
    elif (
            re.search('5.0', grade) or re.search('5.1', grade) or 
            re.search('5.2', grade) or re.search('5.3', grade) or 
            re.search('5.4', grade) or re.search('3rd', grade) or
            re.search('4th', grade) or re.search('5th', grade) or
            re.search('Easy', grade)
    ):
        difficulty = 1

    else:
        difficulty = None

    return difficulty

df['difficulty_sport_trad'] = df.apply(lambda row: difficulty_sport_trad(row), axis=1)
print(df[['difficulty_sport_trad', 'type', 'grade']])

      difficulty_sport_trad             type    grade
0                      13.0  [Sport, Alpine]    5.11c
1                      14.0          [Sport]    5.11d
2                      15.0  [Trad, TR, Aid]    5.12a
3                       8.0          [Sport]  5.10b/c
4                      11.0           [Trad]    5.11a
...                     ...              ...      ...
5980                    8.0           [Trad]    5.10b
5981                   16.0          [Sport]    5.12b
5982                    NaN        [Boulder]   V-easy
5983                    NaN        [Boulder]       V6
5984                    5.0          [Sport]      5.8

[5227 rows x 3 columns]


In [21]:
def num_near_grades(row):
    """Finds the number of routes in close proximity to this route."""
    if row.near_grades == '[]':
        num_grades = 0
        return num_grades
    num_grades_list = row.near_grades.split()
    num_grades = len(num_grades_list)
    return num_grades

df['number_near_grades'] = df.apply(lambda row: num_near_grades(row), axis=1)
df

,name,rating,votes,grade,type,height,pitches,safety,commitment,state,sub_area,photos,comments,near_grades,aid_grade,popularity,combined_difficulty,difficulty_sport_trad,number_near_grades
0,Going Both Ways,3.5,17,5.11c,"[Sport, Alpine]",60.0,1,NaN,NaN,California,Dike Wall,1,0,"['5.11a', '5.11d', '5.11b', '5.11b', '5.10c', ...",NaN,3.433047,5,13.0,13
1,Too Close for Comfort,2.0,2,5.11d,[Sport],40.0,1,NaN,NaN,California,Main Island,0,0,"['5.10a', '5.10a', '5.6', '5.8', '5.10b', '5.1...",NaN,2.098285,5,14.0,6
2,Victim of AIDS,3.1,22,5.12a,"[Trad, TR, Aid]",25.0,1,NaN,NaN,California,Mt Woodson,1,5,"['5.8', '5.7', '5.8', '5.10a']",NaN,3.064994,6,15.0,4
3,Christmas in July,2.4,8,5.10b/c,[Sport],100.0,1,NaN,NaN,California,Rock Creek,1,0,"['5.10', '5.10c', '5.10d', '5.11', '5.10b', '5...",NaN,2.388317,2,8.0,16
4,Bridge of Sighs,2.8,4,5.11a,[Trad],50.0,1,R,NaN,California,Rattlesnake Canyon,0,1,[],NaN,2.698971,4,11.0,0
5,Smooth as a Baby's Ass,1.7,3,V2,[Boulder],13.0,1,NaN,NaN,California,Indian Wave Bou…,1,0,"['V-easy', 'V-easy', 'V0-']",NaN,1.848713,4,NaN,3
6,Crystalline Ceilidh,1.0,1,5.10a/b,"[Sport, Alpine]",45.0,1,NaN,NaN,California,Crystal Crag,0,0,"['5.11d', '5.9', '5.11d']",NaN,1.647427,2,7.0,3
7,Maltese Falcon,3.5,31,5.12a,[Sport],50.0,1,NaN,NaN,California,Main Island,2,1,"['5.8', '5.12d']",NaN,3.462339,6,15.0,2
8,BamBam,1.3,3,5.10a,[Sport],25.0,1,NaN,NaN,California,Little Egypt,0,1,"['5.10a', '5.6', '5.11', '5.11+', '5.9']",NaN,1.548713,2,7.0,5
9,Kids of the Black Hole,1.5,2,V0,[Boulder],12.0,1,NaN,NaN,California,Love Nest Area,2,0,"['V-easy', 'V-easy', 'V4']",NaN,1.764951,2,NaN,3


In [38]:
def diff_near_grades(row):
    """Finds the difference between the average grades of nearby routes
    and the average grade of the current route."""
    if int(row['number_near_grades']) == 0: #If there are no routes nearby
        return None
    else: 
        near_routes = row.near_grades
        near_routes = near_routes.split()
        sum_grades = 0
        for route in near_routes:
            sum_grades = sum_grades + combined_difficulty(route)
    average_nearby = sum_grades / len(near_routes)
    current_grade = combined_difficulty(row.grade)
    difference = abs(current_grade - average_nearby)
    return difference

    
diff_near_grades(df.iloc[100])

df['diff_nearby_grades'] = df.apply(lambda row: diff_near_grades(row), axis=1)
df.head()

,name,rating,votes,grade,type,height,pitches,safety,commitment,state,sub_area,photos,comments,near_grades,aid_grade,popularity,combined_difficulty,difficulty_sport_trad,number_near_grades,diff_nearby_grades
0,Going Both Ways,3.5,17,5.11c,"[Sport, Alpine]",60.0,1,NaN,NaN,California,Dike Wall,1,0,"['5.11a', '5.11d', '5.11b', '5.11b', '5.10c', ...",NaN,3.433047,5,13.0,13,1.307692
1,Too Close for Comfort,2.0,2,5.11d,[Sport],40.0,1,NaN,NaN,California,Main Island,0,0,"['5.10a', '5.10a', '5.6', '5.8', '5.10b', '5.1...",NaN,2.098285,5,14.0,6,2.666667
2,Victim of AIDS,3.1,22,5.12a,"[Trad, TR, Aid]",25.0,1,NaN,NaN,California,Mt Woodson,1,5,"['5.8', '5.7', '5.8', '5.10a']",NaN,3.064994,6,15.0,4,4.000000
3,Christmas in July,2.4,8,5.10b/c,[Sport],100.0,1,NaN,NaN,California,Rock Creek,1,0,"['5.10', '5.10c', '5.10d', '5.11', '5.10b', '5...",NaN,2.388317,2,8.0,16,1.250000
4,Bridge of Sighs,2.8,4,5.11a,[Trad],50.0,1,R,NaN,California,Rattlesnake Canyon,0,1,[],NaN,2.698971,4,11.0,0,NaN


In [21]:
def find_safety_val(row):
    """Creates a new row where safety ratings correspond to numerical values."""
    safety_categorical = row['safety']
    if safety_categorical == 'PG13':
        safety_numerical = 1
    elif safety_categorical == 'R':
        safety_numerical = 2
    elif safety_categorical == 'X':
        safety_numerical = 3
    else:
        return None
    return safety_numerical

df['safety_numerical'] = df.apply(lambda row: find_safety_val(row), axis = 1)
df

,name,rating,votes,grade,type,height,pitches,safety,commitment,state,...,photos,comments,near_grades,aid_grade,popularity,combined_difficulty,difficulty_sport_trad,number_near_grades,diff_nearby_grades,safety_numerical
0,The High Road,2.8,26,V7,[Boulder],25.0,1,NaN,NaN,Nevada,...,1,4,"['V3', 'V7', 'V1', 'V2', 'V9-', 'V7-', 'V5', '...",NaN,2.781291,9,NaN,12,2.500000,NaN
2,White Tigers,2.1,14,5.10b,[Sport],80.0,1,PG13,NaN,Nevada,...,1,1,"['5.10c', '5.9', '5.10a', '5.9']",NaN,2.112990,2,8.0,4,0.250000,1.0
3,Van Betten Arête,3.4,16,V3,[Boulder],10.0,1,NaN,NaN,Nevada,...,2,1,[],NaN,3.334991,5,NaN,0,NaN,NaN
4,Blue Steel Heat,3.0,1,V5,[Boulder],15.0,1,NaN,NaN,California,...,3,1,"['V0', 'V1', 'V1+', 'V4', 'V6', 'V1']",NaN,2.647427,7,NaN,6,2.833333,NaN
6,Paul's Penis,1.0,2,V0,[Boulder],18.0,1,NaN,NaN,California,...,1,0,[],NaN,1.431618,2,NaN,0,NaN,NaN
8,"Not So High, but High Enough",2.0,2,V0,[Boulder],18.0,1,PG13,NaN,California,...,1,0,"['V1', 'V0', 'V2+', 'V1+', 'V-easy', 'V2']",NaN,2.098285,2,NaN,6,0.833333,1.0
9,Dolphin Safe,2.5,31,5.7,"[Trad, Sport]",110.0,1,R,NaN,Nevada,...,1,4,"['5.8', '5.9', '5.3', '5.5', '5.6', '5.8', '5....",NaN,2.493589,2,4.0,12,0.333333,2.0
10,Problem F,0.5,2,V0-,[Boulder],10.0,1,NaN,NaN,California,...,0,0,"['V5+', 'V3', 'V0+', 'V2', 'V2', 'V3', 'V0+', ...",NaN,1.098285,2,NaN,8,2.125000,NaN
13,Through Bein' Cool,2.5,2,5.10c,[Trad],85.0,1,NaN,NaN,California,...,0,1,"['5.9', '5.12a', '5.12b']",NaN,2.431618,3,9.0,3,1.666667,NaN
14,The Itsy Bitsy Spider,3.0,1,5.10,[Trad],400.0,3,NaN,II,Nevada,...,2,0,"['5.8+', '5.7', '5.8', '5.8', '5.8', '5.5', '5...",NaN,2.647427,3,7.0,23,0.956522,NaN


In [22]:
def find_commitment_val(row):
    """Creates a new row where commitment ratings correspond to a numerical value."""
    commitment_categorical = row['commitment']
    if commitment_categorical == 'I':
        commitment_numerical = 1
    elif commitment_categorical == 'II':
        commitment_numerical = 2
    elif commitment_categorical == 'III':
        commitment_numerical = 3
    elif commitment_categorical == 'IV':
        commitment_numerical = 4
    elif commitment_categorical == 'V':
        commitment_numerical = 5
    elif commitment_categorical == 'VI':
        commitment_numerical = 6
    elif commitment_categorical == 'VII':
        commitment_numerical = 7
    else:
        return 1
    return commitment_numerical

df['commitment_numerical'] = df.apply(lambda row: find_commitment_val(row), axis = 1)
df

,name,rating,votes,grade,type,height,pitches,safety,commitment,state,...,comments,near_grades,aid_grade,popularity,combined_difficulty,difficulty_sport_trad,number_near_grades,diff_nearby_grades,safety_numerical,commitment_numerical
0,The High Road,2.8,26,V7,[Boulder],25.0,1,NaN,NaN,Nevada,...,4,"['V3', 'V7', 'V1', 'V2', 'V9-', 'V7-', 'V5', '...",NaN,2.781291,9,NaN,12,2.500000,NaN,1
2,White Tigers,2.1,14,5.10b,[Sport],80.0,1,PG13,NaN,Nevada,...,1,"['5.10c', '5.9', '5.10a', '5.9']",NaN,2.112990,2,8.0,4,0.250000,1.0,1
3,Van Betten Arête,3.4,16,V3,[Boulder],10.0,1,NaN,NaN,Nevada,...,1,[],NaN,3.334991,5,NaN,0,NaN,NaN,1
4,Blue Steel Heat,3.0,1,V5,[Boulder],15.0,1,NaN,NaN,California,...,1,"['V0', 'V1', 'V1+', 'V4', 'V6', 'V1']",NaN,2.647427,7,NaN,6,2.833333,NaN,1
6,Paul's Penis,1.0,2,V0,[Boulder],18.0,1,NaN,NaN,California,...,0,[],NaN,1.431618,2,NaN,0,NaN,NaN,1
8,"Not So High, but High Enough",2.0,2,V0,[Boulder],18.0,1,PG13,NaN,California,...,0,"['V1', 'V0', 'V2+', 'V1+', 'V-easy', 'V2']",NaN,2.098285,2,NaN,6,0.833333,1.0,1
9,Dolphin Safe,2.5,31,5.7,"[Trad, Sport]",110.0,1,R,NaN,Nevada,...,4,"['5.8', '5.9', '5.3', '5.5', '5.6', '5.8', '5....",NaN,2.493589,2,4.0,12,0.333333,2.0,1
10,Problem F,0.5,2,V0-,[Boulder],10.0,1,NaN,NaN,California,...,0,"['V5+', 'V3', 'V0+', 'V2', 'V2', 'V3', 'V0+', ...",NaN,1.098285,2,NaN,8,2.125000,NaN,1
13,Through Bein' Cool,2.5,2,5.10c,[Trad],85.0,1,NaN,NaN,California,...,1,"['5.9', '5.12a', '5.12b']",NaN,2.431618,3,9.0,3,1.666667,NaN,1
14,The Itsy Bitsy Spider,3.0,1,5.10,[Trad],400.0,3,NaN,II,Nevada,...,0,"['5.8+', '5.7', '5.8', '5.8', '5.8', '5.5', '5...",NaN,2.647427,3,7.0,23,0.956522,NaN,2


In [23]:
def find_aid_val(row):
    if (type(row['aid_grade'])) == float:
        return None
    aid_categorical = str(row['aid_grade'])
    if(re.search('A0', aid_categorical)):
        aid_numerical = 1
    elif(re.search('A1', aid_categorical)):
        aid_numerical = 1    
    elif(re.search('A2', aid_categorical)):
        aid_numerical = 2
    elif(re.search('A3', aid_categorical)):
        aid_numerical = 3
    elif(re.search('A4', aid_categorical)):
        aid_numerical = 4
    elif(re.search('A5', aid_categorical)):
        aid_numerical = 5
    else:
        return None
    return aid_numerical

df['aid_numerical'] = df.apply(lambda row: find_aid_val(row), axis = 1)
df

,name,rating,votes,grade,type,height,pitches,safety,commitment,state,...,near_grades,aid_grade,popularity,combined_difficulty,difficulty_sport_trad,number_near_grades,diff_nearby_grades,safety_numerical,commitment_numerical,aid_numerical
0,The High Road,2.8,26,V7,[Boulder],25.0,1,NaN,NaN,Nevada,...,"['V3', 'V7', 'V1', 'V2', 'V9-', 'V7-', 'V5', '...",NaN,2.781291,9,NaN,12,2.500000,NaN,1,NaN
2,White Tigers,2.1,14,5.10b,[Sport],80.0,1,PG13,NaN,Nevada,...,"['5.10c', '5.9', '5.10a', '5.9']",NaN,2.112990,2,8.0,4,0.250000,1.0,1,NaN
3,Van Betten Arête,3.4,16,V3,[Boulder],10.0,1,NaN,NaN,Nevada,...,[],NaN,3.334991,5,NaN,0,NaN,NaN,1,NaN
4,Blue Steel Heat,3.0,1,V5,[Boulder],15.0,1,NaN,NaN,California,...,"['V0', 'V1', 'V1+', 'V4', 'V6', 'V1']",NaN,2.647427,7,NaN,6,2.833333,NaN,1,NaN
6,Paul's Penis,1.0,2,V0,[Boulder],18.0,1,NaN,NaN,California,...,[],NaN,1.431618,2,NaN,0,NaN,NaN,1,NaN
8,"Not So High, but High Enough",2.0,2,V0,[Boulder],18.0,1,PG13,NaN,California,...,"['V1', 'V0', 'V2+', 'V1+', 'V-easy', 'V2']",NaN,2.098285,2,NaN,6,0.833333,1.0,1,NaN
9,Dolphin Safe,2.5,31,5.7,"[Trad, Sport]",110.0,1,R,NaN,Nevada,...,"['5.8', '5.9', '5.3', '5.5', '5.6', '5.8', '5....",NaN,2.493589,2,4.0,12,0.333333,2.0,1,NaN
10,Problem F,0.5,2,V0-,[Boulder],10.0,1,NaN,NaN,California,...,"['V5+', 'V3', 'V0+', 'V2', 'V2', 'V3', 'V0+', ...",NaN,1.098285,2,NaN,8,2.125000,NaN,1,NaN
13,Through Bein' Cool,2.5,2,5.10c,[Trad],85.0,1,NaN,NaN,California,...,"['5.9', '5.12a', '5.12b']",NaN,2.431618,3,9.0,3,1.666667,NaN,1,NaN
14,The Itsy Bitsy Spider,3.0,1,5.10,[Trad],400.0,3,NaN,II,Nevada,...,"['5.8+', '5.7', '5.8', '5.8', '5.8', '5.5', '5...",NaN,2.647427,3,7.0,23,0.956522,NaN,2,NaN


In [24]:
df.aid_grade.unique()

array([nan, 'A4', 'A3', 'A3+', 'A0', 'A2+', 'A2', 'A4-', 'A1', 'A1+',
       'A4+', 'A3-', 'A1-2', 'A5+', 'A2-3', 'A4-5', 'A1-', 'A0-1', 'A0+',
       'A2-', 'A5'], dtype=object)

In [25]:
#Save it as a csv file
df.to_csv('pre_processed.csv')

In [26]:
#Split into 80-20 training data and test data.
eighty_percent = int(len(df.index)*.8)
train = df.iloc[:eighty_percent]
test = df.iloc[eighty_percent:]
print("Shape of new dataframes - {} , {}".format(train.shape, test.shape)) 

train.to_csv('train_data.csv')
test.to_csv('test_data.csv')

Shape of new dataframes - (4181, 23) , (1046, 23)


In [27]:
#Create separate a separate csv file for sport routes
sport = df.copy()

def find_sport(row):
    """Drops rows that aren't purely sport routes."""
    row_type = row['type']
    if (len(row_type) == 1) and (row_type[0] == 'Sport'):
        count = 0
    else:
        sport.drop(index = row.name, inplace = True)

sport.apply(lambda row: find_sport(row), axis = 1)
sport.to_csv('sport_routes.csv')

In [28]:
#Create separate a separate csv file for boulder routes
boulder = df.copy()

def find_boulder(row):
    """Drops rows that aren't purely boulder routes."""
    row_type = row['type']
    if (len(row_type) == 1) and (row_type[0] == 'Boulder'):
        count = 0
    else:
        boulder.drop(index = row.name, inplace = True)

boulder.apply(lambda row: find_boulder(row), axis = 1)
boulder.to_csv('boulder_routes.csv')

In [29]:
#Create separate a separate csv file for trad routes
trad = df.copy()

def find_trad(row):
    """Drops rows that aren't purely trad routes."""
    row_type = row['type']
    if (len(row_type) == 1) and (row_type[0] == 'Trad'):
        count = 0
    else:
        trad.drop(index = row.name, inplace = True)

trad.apply(lambda row: find_trad(row), axis = 1)
trad.to_csv('trad_routes.csv')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5227 entries, 0 to 5983
Data columns (total 23 columns):
name                     5227 non-null object
rating                   5227 non-null float64
votes                    5227 non-null int64
grade                    5227 non-null object
type                     5227 non-null object
height                   5227 non-null float64
pitches                  5227 non-null int64
safety                   666 non-null object
commitment               354 non-null object
state                    5227 non-null object
sub_area                 5227 non-null object
photos                   5227 non-null int64
comments                 5227 non-null int64
near_grades              5227 non-null object
aid_grade                143 non-null object
popularity               5227 non-null float64
combined_difficulty      5227 non-null int64
difficulty_sport_trad    3243 non-null float64
number_near_grades       5227 non-null int64
diff_nearby_grades      